# Data preparation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import requests
import zipfile
from io import BytesIO

# Data download
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
r = requests.get(url)
z = zipfile.ZipFile(BytesIO(r.content))

# Opening CSV
with z.open('AirQualityUCI.csv') as f:
    df = pd.read_csv(f, sep=';', decimal=',', engine='python')
df = df.iloc[:, :-2]  # az utolsó két üres oszlopot töröljük

# Change -200 to NAN
df.replace(-200, pd.NA, inplace=True)

# Choose numeric columns
numeric_cols = df.columns[2:]  # Leave out Date and Time
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Change missing values to median
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

# Choose target column
X = df.drop(columns=['CO(GT)', 'Date', 'Time'])
y = df['CO(GT)']

# First model

In [2]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the first model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prediction
y_pred = model.predict(X_test)

# Results
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

# Feature importance
importances = pd.Series(model.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)
print("Feature importance:")
print(importances)

MSE: 0.26928197631892464
R2: 0.8489502015950271
Feature importance:
PT08.S2(NMHC)    0.498390
NOx(GT)          0.197765
C6H6(GT)         0.154193
NO2(GT)          0.033431
PT08.S1(CO)      0.030252
T                0.019594
PT08.S5(O3)      0.015808
PT08.S3(NOx)     0.013332
PT08.S4(NO2)     0.011326
RH               0.011174
AH               0.010138
NMHC(GT)         0.004599
dtype: float64


# Format Date and Time

In [3]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Time'] = pd.to_datetime(df['Time'], format='%H.%M.%S').dt.hour  

# New feaures
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Delete original Date column
df = df.drop(columns=['Date'])

# Impute missing data with median

In [4]:
from sklearn.impute import SimpleImputer

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

imputer = SimpleImputer(strategy='median')  
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])


# Test several regression models

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# New X and y
X = df.drop(columns=['CO(GT)'])
y = df['CO(GT)']

# Scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modells
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor()
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'MSE': mse, 'R2': r2}

# Results
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by='R2', ascending=False)
print(results_df)


                       MSE        R2
SVR               0.202228  0.886563
KNN               0.221572  0.875712
RandomForest      0.223959  0.874374
GradientBoosting  0.228173  0.872010
Ridge             0.300658  0.831351
LinearRegression  0.300718  0.831317
Lasso             1.674151  0.060910


# Three best models with parameter tuning

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Scaler 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modells and parameters
models_params = {
    'SVR': {
        'model': SVR(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 0.01, 0.1, 1],
            'epsilon': [0.01, 0.1, 0.2]
        },
        'scaled': True
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {
            'n_neighbors': [3,5,7,9],
            'weights': ['uniform', 'distance'],
            'p': [1,2]
        },
        'scaled': True
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200, 500],
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'scaled': False  
    }
}

results = []

for name, mp in models_params.items():
    print(f"=== {name} ===")
    X_tr = X_train_scaled if mp['scaled'] else X_train
    X_te = X_test_scaled if mp['scaled'] else X_test
    
    search = RandomizedSearchCV(
        mp['model'], 
        mp['params'], 
        n_iter=10,  
        cv=3,
        scoring='r2',
        n_jobs=-1,
        random_state=42
    )
    search.fit(X_tr, y_train)
    
    y_pred = search.predict(X_te)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'Best_Params': search.best_params_,
        'MSE': mse,
        'R2': r2
    })
    
results_df = pd.DataFrame(results)
print(results_df)


=== SVR ===


=== KNN ===
=== RandomForest ===
          Model                                        Best_Params       MSE  \
0           SVR       {'gamma': 'scale', 'epsilon': 0.1, 'C': 100}  0.165057   
1           KNN  {'weights': 'distance', 'p': 1, 'n_neighbors': 5}  0.187956   
2  RandomForest  {'n_estimators': 500, 'min_samples_split': 2, ...  0.216820   

         R2  
0  0.907413  
1  0.894569  
2  0.878378  
